In [4]:
pip install gpxplotter

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.7/443.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.7 MB/s eta 0:00:00
  Created wheel for vincent: filename=vincent-0.4.4-py3-none-any.whl size=35149 sha256=2c1119f1c8e5adebb7c9dda550a19cacba61e016e6350b7365028452996b832d
  Stored in directory: /root/.cache/pip/wheels/18/a5/32/c4e033e195f545fa2682419991bc001d6f5d78159152917999
Successfully built vincent


In [1]:
from google.colab import drive
drive.mount('/content/drive')                           #mounting my google drive on colab

Mounted at /content/drive


In [2]:
!ls "/content/drive/My Drive/Classroom/watch8/workout-routes/"                 #checking directory under watch8 folder

route_2023-05-26_7.41pm.gpx   route_2023-07-26_5.54pm.gpx   route_2024-03-07_5.57pm.gpx
route_2023-05-27_5.55pm.gpx   route_2023-07-26_6.07pm.gpx   route_2024-03-08_7.10pm.gpx
route_2023-05-27_6.10am.gpx   route_2023-07-26_7.10pm.gpx   route_2024-03-10_3.21pm.gpx
route_2023-05-27_6.20am.gpx   route_2023-07-27_7.02pm.gpx   route_2024-03-12_5.35pm.gpx
route_2023-05-27_6.41am.gpx   route_2023-07-28_4.19pm.gpx   route_2024-03-13_5.34pm.gpx
route_2023-05-27_7.31am.gpx   route_2023-07-28_5.50pm.gpx   route_2024-03-14_5.34pm.gpx
route_2023-05-28_6.10am.gpx   route_2023-07-29_1.54pm.gpx   route_2024-03-15_5.38pm.gpx
route_2023-05-28_6.30am.gpx   route_2023-07-31_5.13am.gpx   route_2024-03-16_4.36pm.gpx
route_2023-05-28_6.56am.gpx   route_2023-07-31_5.35am.gpx   route_2024-03-18_5.08pm.gpx
route_2023-05-29_4.56pm.gpx   route_2023-07-31_5.57am.gpx   route_2024-03-19_4.46pm.gpx
route_2023-05-29_5.17pm.gpx   route_2023-08-04_5.46am.gpx   route_2024-03-20_4.31pm.gpx
route_2023-05-29_6.33pm.gpx   ro

In [35]:
from gpxplotter import create_folium_map, read_gpx_file, add_segment_to_map
import folium
line_options = {"weight": 4}
mapper = create_folium_map(tiles="openstreetmap")
for track in read_gpx_file("/content/drive/My Drive/Classroom/watch8/workout-routes/route_2024-06-30_5.51am.gpx"):
    for i, segment in enumerate(track["segments"]):
        add_segment_to_map(
            mapper,
            segment,
            color_by="Velocity / km/h",
            cmap="viridis",
            line_options=line_options,
        )
#mapper

In [6]:
import PIL
from PIL.ExifTags import TAGS, GPSTAGS
import datetime


# Create a method to get coordinates from a photo:
def get_lat_lon(imagefile):
    image = PIL.Image.open(imagefile)
    exif = image.getexif()
    exif_info = {TAGS.get(key, key): val for key, val in exif.items()}
    # Get the GPSInfo:
    for key, value in TAGS.items():
        if value == "GPSInfo":
            exif_info[value] = exif.get_ifd(key)
            break
    gps_info = {}
    for key, val in exif_info["GPSInfo"].items():
        gps_info[GPSTAGS.get(key, key)] = val
    # Convert to decimal latitude/longitude:  degree to lat/lon ()
    deg, minutes, seconds = gps_info["GPSLatitude"]
    latitude = deg + minutes / 60.0 + seconds / 3600.0
    if "GPSLatitudeRef" == "S":
        latitude *= -1
    deg, minutes, seconds = gps_info["GPSLongitude"]
    longitude = deg + minutes / 60.0 + seconds / 3600.0
    if "GPSLongitudeRef" == "W":
        longitude *= -1
    # Turn time into datetime:
    time = datetime.datetime.strptime(
        exif_info["DateTime"], "%Y:%m:%d %H:%M:%S"
    )
    return latitude, longitude, time

In [27]:
!ls "/content/drive/My Drive/Classroom/watch8/geotagged_pics/"                 #checking directory under geotagged pics folder

1.jpg  2.jpg


In [36]:
info = {}
for filename in ("/content/drive/MyDrive/Classroom/watch8/geotagged_pics/1.jpg", "/content/drive/MyDrive/Classroom/watch8/geotagged_pics/2.jpg" , "/content/drive/MyDrive/Classroom/watch8/geotagged_pics/3.jpg"):
    lat, lon, time = get_lat_lon(filename)
    info[filename] = {"latlon": (lat, lon), "time": time}

In [45]:
for key, val in info.items():
    marker = folium.Marker(
        location=val["latlon"],
        tooltip=f"You took picture right here  {key} iPhone14pro...",
        icon=folium.Icon(icon="camera", color="green"),
    )
    marker.add_to(mapper)
boundary = mapper.get_bounds()
mapper.fit_bounds(boundary, padding=(2, 2))
#mapper.zoom_control.add_to(mapper)
#mapper.show_in_browser
#mapper.save("map.html")
mapper